In [20]:
# =============================================================================
# Imports
# =============================================================================
import os
import sys
import json
import csv


In [21]:
# =============================================================================
# Declares
# =============================================================================
# Empirical 13-datasets

# BUSTED
BUSTEDS_DIR = "/Users/user/Documents/BS-MH/analysis/13-datasets/BUSTEDS"
BUSTEDS_MH_DIR = "/Users/user/Documents/BS-MH/analysis/13-datasets/BUSTEDS-MH"
BUSTEDS_OUTPUT_CSV = "PROCESSED_PairedAnalysis_13datasets_BUSTEDS-MH_vs_BUSTEDS.csv"

# aBSREL
aBSREL_DIR = "/Users/user/Documents/BS-MH/analysis/13-datasets/aBSREL"
aBSREL_MH_DIR = "/Users/user/Documents/BS-MH/analysis/13-datasets/aBSREL-MH"
aBSREL_OUTPUT_CSV = "PROCESSED_PairedAnalysis_13datasets_aBSREL-MH_vs_aBSREL.csv"


In [22]:
def read_json_BUSTEDS(filename):
    print("# Reading:", filename)
    if os.stat(filename).st_size == 0: return 9999991 
    with open(filename, "r") as fh:
        json_data = json.load(fh)
    fh.close()
    p_value = json_data["test results"]["p-value"]
    BASE_Unconstrained_cAIC = json_data["fits"]["Unconstrained model"]["AIC-c"]
    BASE_Unconstrained_log_L = json_data["fits"]["Unconstrained model"]["Log Likelihood"]
    print("\tBaseline model (BUSTEDS) p-value =", p_value)
    print("\tBaseline model (BUSTEDS) Unconstrained cAIC =", BASE_Unconstrained_cAIC)
    print("\tBaseline model (BUSTEDS) Unconstrained logL =", BASE_Unconstrained_log_L)
    return p_value, BASE_Unconstrained_cAIC, BASE_Unconstrained_log_L
#end method

def read_json_BUSTEDS_MH(filename_MH):
    print("# Reading:", filename_MH)
    if os.stat(filename_MH).st_size == 0: return 9999990
    with open(filename_MH, "r") as fh:
        json_data = json.load(fh)
    fh.close()
    # Part 1 - MG94 with double and triple instantaneous substitutions"
    p_value = json_data["test results"]["p-value"]
    MH_cAIC = json_data["fits"]["MG94 with double and triple instantaneous substitutions"]["AIC-c"]
    MH_log_L = json_data["fits"]["MG94 with double and triple instantaneous substitutions"]["Log Likelihood"]
    MH_dNdS = json_data["fits"]["MG94 with double and triple instantaneous substitutions"]["Rate Distributions"]["non-synonymous/synonymous rate ratio for *test*"]
    MH_DH_rate = json_data["fits"]["MG94 with double and triple instantaneous substitutions"]["Rate Distributions"]["rate at which 2 nucleotides are changed instantly within a single codon"]
    MH_TH_rate = json_data["fits"]["MG94 with double and triple instantaneous substitutions"]["Rate Distributions"]["rate at which 3 nucleotides are changed instantly within a single codon"]
    MH_TH_SI_rate = json_data["fits"]["MG94 with double and triple instantaneous substitutions"]["Rate Distributions"]["rate at which 3 nucleotides are changed instantly within a single codon between synonymous codon islands"]
    # Part 2 - "Unconstrained model"
    MH_Unconstrained_cAIC = json_data["fits"]["Unconstrained model"]["AIC-c"]
    MH_Unconstrained_log_L = json_data["fits"]["Unconstrained model"]["Log Likelihood"]
    #UN_SRV_0 = json_data["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]
    #UN_SRV_1 = json_data["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]
    #UN_SRV_2 = json_data["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]
    MH_Unconstrained_DH_rate = json_data["fits"]["Unconstrained model"]["rate at which 2 nucleotides are changed instantly within a single codon"]
    MH_Unconstrained_TH_rate = json_data["fits"]["Unconstrained model"]["rate at which 3 nucleotides are changed instantly within a single codon"]
    MH_Unconstrained_TH_SI_rate = json_data["fits"]["Unconstrained model"]["rate at which 3 nucleotides are changed instantly within a single codon between synonymous codon islands"]
    #Report on what we found.
    print("\tNovel model (BUSTEDS-MH) p-value =", p_value)
    print("\tNovel model (BUSTEDS-MH) cAIC =", MH_Unconstrained_cAIC)
    print("\tNovel model (BUSTEDS-MH) logL =", MH_Unconstrained_log_L)
    return p_value, MH_cAIC, MH_log_L, MH_dNdS, MH_DH_rate, MH_TH_rate, MH_TH_SI_rate, MH_Unconstrained_cAIC, MH_Unconstrained_log_L, MH_Unconstrained_DH_rate, \
                MH_Unconstrained_TH_rate, \
                MH_Unconstrained_TH_SI_rate
#end method

def process_BUSTEDS(filename_MH, filename):
    global BUSTEDS_OUTPUT_CSV
    p_value_BUSTEDS, BASE_Unconstrained_cAIC, BASE_Unconstrained_log_L = read_json_BUSTEDS(filename)
    
    p_value_BUSTEDS_MH, MH_MG94_cAIC, MH_MG94_log_L, MH_MG94_dNdS, MH_MG94_DH_rate, MH_MG94_TH_rate, MH_MG94_TH_SI_rate, MH_Unconstrained_cAIC, MH_Unconstrained_log_L, MH_Unconstrained_DH_rate, \
                MH_Unconstrained_TH_rate, \
                MH_Unconstrained_TH_SI_rate  = read_json_BUSTEDS_MH(filename_MH)
    
    #Calculate aBSREL-MH minus aBSREL
    #delta_cAIC = float(cAIC_aBSREL_MH) - float(cAIC_aBSREL)
    #print("\t### RESULT", delta_cAIC)
    delta_p_value = float(p_value_BUSTEDS_MH) - float(p_value_BUSTEDS)
    #Check for errors
    if p_value_BUSTEDS == 9999990 or p_value_BUSTEDS_MH == 9999991: return 1
    #Output to csv
    with open(BUSTEDS_OUTPUT_CSV, 'a', newline='') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerow([filename_MH.split("/")[-1].replace(".BUSTEDS.json", ""), str(p_value_BUSTEDS_MH), str(p_value_BUSTEDS), str(delta_p_value),
                             BASE_Unconstrained_cAIC, BASE_Unconstrained_log_L, \
                             MH_MG94_cAIC, MH_MG94_log_L, \
                                 MH_MG94_dNdS, MH_MG94_DH_rate, MH_MG94_TH_rate, MH_MG94_TH_SI_rate, \
                                 MH_Unconstrained_cAIC, MH_Unconstrained_log_L, \
                                     MH_Unconstrained_DH_rate, \
                                     MH_Unconstrained_TH_rate, \
                                     MH_Unconstrained_TH_SI_rate
                             ])
    csvfile.close()
    #end with
#end method

In [23]:
# =============================================================================
# Main subroutine
# =============================================================================
print("# Init ")
print("# Program is in:", os.getcwd())
print("# Saving output to:", BUSTEDS_OUTPUT_CSV)

#Empty out the output CSV
with open(BUSTEDS_OUTPUT_CSV, 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(["Filename", "pvalue_BUSTEDSMH", "pvalue_BUSTEDS", "delta p-values", \
                         "BASE_Unconstrained_cAIC", "BASE_Unconstrained_log_L",
                         "MH_MG94x(2+3)_cAIC", "MH_MG94x(2+3)_log_L", "MH_MG94x(2+3)_dNdS", "MH_MG94x(2+3)_DH_rate", "MH_MG94x(2+3)_TH_rate", "MH_MG94x(2+3)_TH_SI_rate", \
                         "MH_Unconstrained_cAIC", "MH_Unconstrained_log_L", \
                         "MH_Unconstrained_DH_rate", "MH_Unconstrained_TH_rate", "MH_Unconstrained_TH_SI_rate"])
csvfile.close()
#end with
    
print("# Processing ", BUSTEDS_DIR)
print("# Processing ", BUSTEDS_MH_DIR)
BASE_tag = ""
MH_tag = ""

BUSTEDS_DIR_FILES = [os.path.join(BUSTEDS_DIR, file.name) for file in os.scandir(BUSTEDS_DIR) if file.name.endswith(".BUSTEDS.json")]
BUSTEDS_MH_DIR_FILES = [os.path.join(BUSTEDS_MH_DIR, file.name) for file in os.scandir(BUSTEDS_MH_DIR) if file.name.endswith(".BUSTEDS-MH.json")]
print("# Number of BUSTEDS results:", len(BUSTEDS_DIR_FILES))
print("# Number of BUSTEDS-MH results:", len(BUSTEDS_MH_DIR_FILES))

# Loop over files in aBSREL-MH_DIR
matches_count = 0
for n, file in enumerate(BUSTEDS_MH_DIR_FILES):
    if os.stat(file).st_size == 0: 
        print("# Encountered empty file:", file)
        continue
    #end if
    
    #print("# Searching for match for:", file.split("/")[-1])
    print("#", n+1, "Searching for match for:", file)
    
    NEXUS_FILENAME = file.split("/")[-1].replace(".BUSTEDS-MH.json", "") 
    
    BUSTEDS_VERSION = os.path.join(BUSTEDS_DIR, NEXUS_FILENAME) + ".BUSTEDS.json"
    
    if os.stat(file).st_size > 0: 
        print("# Checking for BUSTEDS output for:", BUSTEDS_VERSION)
    else:
        continue
    #end if
    
    # Does it match with a file in aBSREL_DIR?, if soo..
    if BUSTEDS_VERSION in BUSTEDS_DIR_FILES:
        # process
        matches_count += 1
        process_BUSTEDS(file, BUSTEDS_VERSION)
    else:
        # dont process, but record this.
        pass
    # end if
    print()
#end for

print("# Number of matches:", matches_count)
             


# Init 
# Program is in: /Users/user/Documents/BS-MH/scripts
# Saving output to: PROCESSED_PairedAnalysis_13datasets_BUSTEDS-MH_vs_BUSTEDS.csv
# Processing  /Users/user/Documents/PRELIM_2020/14-datasets/BUSTEDS
# Processing  /Users/user/Documents/PRELIM_2020/14-datasets/BUSTEDS-MH
# Number of BUSTEDS results: 13
# Number of BUSTEDS-MH results: 13
# 1 Searching for match for: /Users/user/Documents/BS-MH/analysis/13-datasets/BUSTEDS-MH/InfluenzaA.nex.BUSTEDS-MH.json
# Checking for BUSTEDS output for: /Users/user/Documents/BS-MH/analysis/13-datasets/BUSTEDS/InfluenzaA.nex.BUSTEDS.json
# Reading: /Users/user/Documents/BS-MH/analysis/13-datasets/BUSTEDS/InfluenzaA.nex.BUSTEDS.json
	Baseline model (BUSTEDS) p-value = 0.1047127986361128
	Baseline model (BUSTEDS) Unconstrained cAIC = 23242.49538542873
	Baseline model (BUSTEDS) Unconstrained logL = -11051.42889583376
# Reading: /Users/user/Documents/BS-MH/analysis/13-datasets/BUSTEDS-MH/InfluenzaA.nex.BUSTEDS-MH.json
	Novel model (BUSTEDS-MH) p

In [18]:
# =============================================================================
# End of file
# =============================================================================